In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import time
import openai
from dotenv import load_dotenv

from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup as bs

from method.ours.utils import create_driver
from method.ours.parse import parse_form
from method.ours.prompts import get_form_context
from method.ours.generation import (
    generate_constraints_for_input_groups,
    generate_value_for_input_group,
    generate_values_for_input_groups,
    fill_form_with_value_table,
    submit_form
)
from method.ours.feedback import (
    get_local_feedback,
    get_global_feedback
)

In [3]:
load_dotenv()
openai.api_key = os.getenv("OPENAI_API_KEY")

# Global Variables
HEADLESS = False
TEXT_EMBEDDING_METHOD = 'ADA' # ['ADA', 'WORD2VEC', 'SPACY']
GRAPH_EMBEDDING_METHOD = 'NODE2VEC' # ['NODE2VEC', 'GCN']

In [4]:
driver = create_driver(HEADLESS)
driver.get('https://www.aircanada.com/ca/en/aco/home.html')

In [6]:
form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [7]:
for element in form.find_elements(By.TAG_NAME, 'input'):
    try:
        element.clear()
    except:
        pass

# Processing

In [8]:
html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')
relation_graph, input_groups = parse_form(
    driver,
    form,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/340 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 34/34 [00:09<00:00,  3.61it/s]
/Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/openai/embeddings_utils.py:68: RuntimeWarning: invalid value encountered in scalar divide
  return np.dot(a, b) / (np.linalg.norm(a) * np.linalg.norm(b))


# Generation

In [9]:
value_table = generate_constraints_for_input_groups(input_groups)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774328,
  "id": "chatcmpl-7ZpdYEfykp1klu9UQ1Nt96S4REJHO",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 615,
    "total_tokens": 632
  }
}
  Total running cost: $0.058
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant informat

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774335,
  "id": "chatcmpl-7Zpdfwazr1lS7nAuUJg3NVOtNKewc",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 643,
    "total_tokens": 672
  }
}
  Total running cost: $0.118
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input f

In [10]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_R\" name=\"tripTyp

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Toronto",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774368,
  "id": "chatcmpl-7ZpeCdf4Sdi6PJU8rRgFyv2xKpqx9",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 283,
    "total_tokens": 284
  }
}
  Total running cost: $0.206
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, an

In [11]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (567, 597), x: (219, 319)
R
<input>value is O</input> at y: (567, 597), x: (334, 421)
O
<input>value is M</input> at y: (567, 597), x: (437, 591)
M
<input></input> at y: (574, 594), x: (652, 905)
true
<input></input> at y: (628, 678), x: (198, 466)
Vancouver
<input></input> at y: (628, 678), x: (536, 804)
Toronto
<input>DD/MM</input> at y: (628, 678), x: (833, 958)
15/07
<input>DD/MM</input> at y: (628, 678), x: (978, 1093)
20/07


In [16]:
submit_form(driver, input_groups)

# Feedback with Diff

In [17]:
new_html = driver.find_element(By.TAG_NAME, 'body').get_attribute('outerHTML')

In [18]:
global_feedback = get_global_feedback(html, new_html)

In [19]:
global_feedback

["We're sorry, we were not able to process your request. Please try again, or contact usOpens in a new tab for assistance.7"]

# Feedback

In [20]:
new_form = driver.find_elements(By.TAG_NAME, 'form')[1]

In [21]:
new_relation_graph, new_input_groups = parse_form(
    driver,
    new_form,
    prev_relation_graph=relation_graph,
    TEXT_EMBEDDING_METHOD=TEXT_EMBEDDING_METHOD
)

Computing transition probabilities:   0%|          | 0/366 [00:00<?, ?it/s]

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 38/38 [00:10<00:00,  3.54it/s]


In [22]:
# diff = relation_graph.diff(new_relation_graph)

In [23]:
value_table = generate_constraints_for_input_groups(
    new_input_groups,
    value_table,
    global_feedback
)

  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant information section. You must choose your constraints in the format of our modified version of the Jest library in JavaScript. The list of functions in this modified format are:\n1. toBeEqual(value) # the input field value is exactly equal to the given value\n2. toHaveLengthCondition(condition, value) # the length of the input field value matches the given condition\n3. toBeTruthy() # the input field value is truthy and not empty (not false, 0, '', null, undefined, or NaN)\n4. toHaveCompareCondit

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_tripTypeSelector_M')).toBeEqual('M')",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774468,
  "id": "chatcmpl-7ZpfowVRfgFcmn0ozgH3vCXaS9KNa",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 17,
    "prompt_tokens": 730,
    "total_tokens": 747
  }
}
  Total running cost: $0.289
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input field in question, not those in the relevant informat

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "expect(field('bkmgFlights_destination_trip_1'))\n.toBeTruthy()\n.not.toBeEqual(field('bkmgFlights_origin_trip_1'))",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774477,
  "id": "chatcmpl-7ZpfxpM3rhiDCahwrnCJnvC68N6j0",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 29,
    "prompt_tokens": 696,
    "total_tokens": 725
  }
}
  Total running cost: $0.356
  Prompt: [
  {
    "role": "system",
    "content": "Instructions:\nYour task is to generate a set of constraints for web form fields. Your decisions must be made independently without seeking user assistance or additional information. If there are multiple ways to express constraints, use the least number of constraints to describe them. Only generate the constraints and refrain from explaining your answers. Only generate constraints for the input f

In [24]:
value_table = generate_values_for_input_groups(input_groups, value_table)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_tripTypeSelector_R\" name=\"tripTyp

  Response: {
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "message": {
        "content": "Toronto",
        "role": "assistant"
      }
    }
  ],
  "created": 1688774497,
  "id": "chatcmpl-7ZpgH6bQf0kDc6ic3pStD4ISUNt65",
  "model": "gpt-4-0613",
  "object": "chat.completion",
  "usage": {
    "completion_tokens": 1,
    "prompt_tokens": 297,
    "total_tokens": 298
  }
}
  Total running cost: $0.451
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, an

In [25]:
fill_form_with_value_table(driver, value_table, input_groups)

<input>value is R</input> at y: (649, 679), x: (219, 319)
R
<input>value is O</input> at y: (649, 679), x: (334, 421)
O
<input>value is M</input> at y: (649, 679), x: (437, 591)
M
<input></input> at y: (656, 676), x: (652, 905)
true
<input></input> at y: (710, 760), x: (198, 466)
Vancouver
<input></input> at y: (710, 760), x: (536, 804)
Toronto
<input>DD/MM</input> at y: (710, 760), x: (833, 958)
15/07
<input>DD/MM</input> at y: (710, 760), x: (978, 1093)
20/07


In [27]:
submit_form(driver, input_groups)

# Constraint Negation

In [28]:
form_context = get_form_context(input_groups)

In [29]:
value_table.entries['bkmgFlights_origin_trip_1'].constraints[1].to_prompt_string()

'input field should be alphabetic'

In [30]:
for i in range(len(value_table.entries['bkmgFlights_origin_trip_1'].constraints)):
    value_table_copy = value_table.copy()
    
    # TODO: implement a flip function for constraints
    value_table_copy.entries['bkmgFlights_origin_trip_1'].constraints = [
        value_table.entries['bkmgFlights_origin_trip_1'].constraints[i]
    ]
    value_table_copy.entries['bkmgFlights_origin_trip_1'].constraints[0].flip_negative()
    
    
    input_group = value_table_copy.entries['bkmgFlights_origin_trip_1'].input_group
    
    value_table_copy = generate_value_for_input_group(
        input_group,
        value_table_copy,
        form_context=form_context
    )
    
    fill_form_with_value_table(driver, value_table_copy, input_groups)
    
    time.sleep(10)
    
    submit_form(driver, input_groups)
    
    time.sleep(10)

  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with points , From, To, Departure, Return\n    We are generating filling values for the following input field:\n    input: <input id=\"bkmgFlights_origin_trip_1\" name=\"bkmgFlights_

<input></input> at y: (710, 760), x: (536, 804)
Toronto
<input>DD/MM</input> at y: (710, 760), x: (833, 958)
15/07
<input>DD/MM</input> at y: (710, 760), x: (978, 1093)
20/07
  Prompt: [
  {
    "role": "system",
    "content": "Your task is to generate a value for a web form field based on the form field information and a set of constraints on the field. Your decisions must be made independently without seeking user assistance or additional information. For each user prompt, you need to generate one value that satisfies the constraints while keeping in mind the nature of the input from the available information. Only generate value and refrain from explaining your answers. Only generate value for the input field in question, and not the ones in the relevant information section."
  },
  {
    "role": "user",
    "content": "The following are all the labels in the form, which provide context for the functionality of the form:  Round-trip ,  One-way ,  Multi-city/Stopover ,  Book with po